In [2]:
#Importing packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#Setting large figure size for Seaborn
sns.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":18})
import dill
import os
import shutil

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
#Importing helpers/labels.py from the cityscrapesScripts github page (https://github.com/mcordts/cityscapesScripts)
from labels import *
id2label[9].category 

'flat'

In [2]:
category = id2label[2].category
print("Category of label with ID '{id}': {category}".format( id=2, category=category ))

Category of label with ID '2': void
